<a href="https://colab.research.google.com/github/CobiELF/ltl-seq2seq/blob/master/585_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install allennlp

In [0]:
#python
import itertools
import os
#pytorch
import torch
import torch.optim as optim
from torch import nn
#numpy
import numpy as np
#allenNLP.data
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.vocabulary import Vocabulary
#allenNLP.nn
from allennlp.nn.activations import Activation
#allenNLP.models
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
#allenNLP.modules
from allennlp.modules.attention import LinearAttention, AdditiveAttention
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
#allenNLP.predictors
from allennlp.predictors import SimpleSeq2SeqPredictor
#allenNLP.training
from allennlp.training.trainer import Trainer
from allennlp.training.checkpointer import Checkpointer

#hyperparameters
SRC_DIM = 256
HID_DIM = 256
TAR_DIM = 128
SRC_TKN = "source_tokens"
NUM_TKN = "num_tokens"
TAR_TKN = "target_tokens"
BATCH = 32
MAX_EPOCHS = 100
MAX_DECODE = 128
BEAM_WIDTH = 15
CUDA_DEVICE = 0

In [0]:
class LTLSeq2SeqReader(Seq2SeqDatasetReader):
  def __init__(self, tsv_url):
    self.tsv_url = tsv_url
    self.read()
  
  def read(self):
    reader = Seq2SeqDatasetReader(
      source_tokenizer = WordTokenizer(),
      target_tokenizer = WordTokenizer(),
      source_token_indexers = {'tokens': SingleIdTokenIndexer()},
      target_token_indexers = {'tokens': SingleIdTokenIndexer(namespace=TAR_TKN)}
    )
    dataset = reader.read(self.tsv_url)
    self.reader = reader
    self.dataset = dataset
    return

class LTLSeq2SeqModel():
  def __init__(self, train_reader, valid_reader):
    self.train_dataset = train_reader.dataset
    self.validation_dataset = valid_reader.dataset
    vocab = Vocabulary.from_instances(self.train_dataset + self.validation_dataset)
    src_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                              embedding_dim=SRC_DIM)
    src_embedder = BasicTextFieldEmbedder({"tokens": src_embedding})
    encoder = PytorchSeq2SeqWrapper(torch.nn.LSTM(SRC_DIM, HID_DIM, batch_first=True))
    bahdanau = AdditiveAttention(HID_DIM, HID_DIM)
    model = SimpleSeq2Seq(vocab, src_embedder, encoder, MAX_DECODE,
                          target_embedding_dim=TAR_DIM,
                          target_namespace=TAR_TKN,
                          beam_size=BEAM_WIDTH,
                          attention=bahdanau,
                          use_bleu=False)
    iterator = BucketIterator(batch_size=BATCH, sorting_keys=[(SRC_TKN, NUM_TKN)])
    iterator.index_with(vocab)
    # train model
    optimizer = optim.Adam(model.parameters())
    model = model.cuda(CUDA_DEVICE)
    trainer = Trainer(model=model,
                      iterator=iterator,
                      optimizer=optimizer,
                      train_dataset=self.train_dataset,
                      validation_dataset=self.validation_dataset,
                      cuda_device=CUDA_DEVICE,
                      #patience=20,
                      num_epochs=MAX_EPOCHS)
    self.metrics = trainer.train()
    self.model = model
    self.early_stop = self.metrics['epoch'] < MAX_EPOCHS - 1
    return
    

In [0]:
def evaluate(decoder, eval_reader, outfile="results.tsv", k=BEAM_WIDTH):
  reader = eval_reader.reader
  eval_dataset = eval_reader.dataset
  fp = open(outfile, "w")
  #fp.write("SRC \t GND \t PRE \t RNK\n")
  best_correct = 0
  in_top_k = 0
  predictor = SimpleSeq2SeqPredictor(decoder.model, reader)
  for instance in eval_dataset:
      output_dict = predictor.predict_instance(instance)
      preds = output_dict['predictions']
      src_tokens = instance.fields[SRC_TKN].tokens
      gnd_tokens = instance.fields[TAR_TKN].tokens
      prd_tokens = output_dict['predicted_tokens']
      src_str = ""
      gnd_str = ""
      prd_str = ""
      # get source string
      for tok in src_tokens[1:-1]:
        src_str += tok.text + " "
      src_str = src_str[:-1]
      # get ground truth string
      for tok in gnd_tokens[1:-1]:
        gnd_str += tok.text
      # get predicted strings
      rank = BEAM_WIDTH+1
      for i in range(0, BEAM_WIDTH):
        best_found = False
        prd_str = ""
        output_dict['predictions'] = np.asarray([preds[i]])
        decoded = decoder.model.decode(output_dict)['predicted_tokens'][0]
        for char in decoded:
          prd_str += char
        fp.write(prd_str + "\t")
        if gnd_str == prd_str:
          if i==0: best_correct += 1
          if not best_found:
            in_top_k += 1
            rank = i+1
            best_found = True
      fp.write(src_str + "\t" + gnd_str + "\t" + str(rank) + "\n")

  acc = (best_correct/len(eval_dataset)) * 100
  acc_str = "ACC: %" + str(acc)
  topk_acc = (in_top_k/len(eval_dataset)) * 100
  topk_acc_str = "TOPK: %" + str(topk_acc)
  print(acc_str)
  print(topk_acc_str)
  fp.write(acc_str)
  fp.write(topk_acc_str)
  fp.close()
  return acc, topk_acc

In [0]:
train_reader_1 = LTLSeq2SeqReader("https://raw.githubusercontent.com/CobiELF/ltl-amdp/master/mod/TRAIN_1.tsv")
test_reader_1 = LTLSeq2SeqReader("https://raw.githubusercontent.com/CobiELF/ltl-amdp/master/mod/TEST_1.tsv")
model_1 = LTLSeq2SeqModel(train_reader_1, test_reader_1)

print(str(model_1.metrics))


0it [00:00, ?it/s]
  0%|          | 0/48443 [00:00<?, ?B/s]
4948it [00:04, 1085.18it/s]
0it [00:00, ?it/s]
  0%|          | 0/12917 [00:00<?, ?B/s]
1237it [00:01, 797.11it/s]
100%|██████████| 6185/6185 [00:00<00:00, 56980.07it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 1.3342 ||: 100%|██████████| 155/155 [00:05<00:00, 27.01it/s]
loss: 0.8104 ||: 100%|██████████| 39/39 [00:01<00:00, 33.20it/s]
loss: 0.7036 ||: 100%|██████████| 155/155 [00:05<00:00, 27.32it/s]
loss: 0.6298 ||: 100%|██████████| 39/39 [00:01<00:00, 31.10it/s]
loss: 0.5786 ||: 100%|██████████| 155/155 [00:05<00:00, 30.45it/s]
loss: 0.5455 ||: 100%|██████████| 39/39 [00:01<00:00, 35.82it/s]
loss: 0.4808 ||: 100%|██████████| 155/155 [00:05<00:00, 28.90it/s]
loss: 0.4214 ||: 100%|██████████| 39/39 [00:01<00:00, 33.04it/s]
loss: 0.3729 ||: 100%|██████████| 155/155 [00:05<00:00, 28.43it/s]
loss: 0.3268 ||: 100%|██████████| 39/39 [00:01<00:00, 34.33it/s]
loss: 

{'best_epoch': 30, 'peak_cpu_memory_MB': 2660.468, 'peak_gpu_0_memory_MB': 847, 'training_duration': '0:12:36.760386', 'training_start_epoch': 0, 'training_epochs': 99, 'epoch': 99, 'training_loss': 0.012792183655901387, 'training_cpu_memory_MB': 2660.468, 'training_gpu_0_memory_MB': 847, 'validation_loss': 0.024851933676846107, 'best_validation_loss': 0.01830147780949632}


In [0]:
train_reader_2 = LTLSeq2SeqReader("https://raw.githubusercontent.com/CobiELF/ltl-amdp/master/mod/TRAIN_2.tsv")
test_reader_2 = LTLSeq2SeqReader("https://raw.githubusercontent.com/CobiELF/ltl-amdp/master/mod/TEST_2.tsv")
model_2 = LTLSeq2SeqModel(train_reader_2, test_reader_2)

print(str(model_2.metrics))

0it [00:00, ?it/s]
  0%|          | 0/48380 [00:00<?, ?B/s]
4948it [00:04, 1167.50it/s]
0it [00:00, ?it/s]
  0%|          | 0/12999 [00:00<?, ?B/s]
1237it [00:01, 898.60it/s]
100%|██████████| 6185/6185 [00:00<00:00, 63229.74it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 1.3707 ||: 100%|██████████| 155/155 [00:05<00:00, 28.42it/s]
loss: 0.7630 ||: 100%|██████████| 39/39 [00:01<00:00, 33.51it/s]
loss: 0.7237 ||: 100%|██████████| 155/155 [00:05<00:00, 30.36it/s]
loss: 0.6581 ||: 100%|██████████| 39/39 [00:01<00:00, 31.19it/s]
loss: 0.6341 ||: 100%|██████████| 155/155 [00:05<00:00, 29.64it/s]
loss: 0.5826 ||: 100%|██████████| 39/39 [00:01<00:00, 33.59it/s]
loss: 0.5393 ||: 100%|██████████| 155/155 [00:05<00:00, 30.15it/s]
loss: 0.5101 ||: 100%|██████████| 39/39 [00:01<00:00, 36.99it/s]
loss: 0.4400 ||: 100%|██████████| 155/155 [00:05<00:00, 28.96it/s]
loss: 0.3992 ||: 100%|██████████| 39/39 [00:01<00:00, 37.27it/s]
loss: 

{'best_epoch': 29, 'peak_cpu_memory_MB': 2711.512, 'peak_gpu_0_memory_MB': 849, 'training_duration': '0:11:45.876641', 'training_start_epoch': 0, 'training_epochs': 99, 'epoch': 99, 'training_loss': 0.013075615739627768, 'training_cpu_memory_MB': 2711.512, 'training_gpu_0_memory_MB': 849, 'validation_loss': 0.030146374756161373, 'best_validation_loss': 0.019016137548113383}


In [0]:
train_reader_3 = LTLSeq2SeqReader("https://raw.githubusercontent.com/CobiELF/ltl-amdp/master/mod/TRAIN_3.tsv")
test_reader_3 = LTLSeq2SeqReader("https://raw.githubusercontent.com/CobiELF/ltl-amdp/master/mod/TEST_3.tsv")
model_3 = LTLSeq2SeqModel(train_reader_3, test_reader_3)

print(str(model_3.metrics))

0it [00:00, ?it/s]
  0%|          | 0/48558 [00:00<?, ?B/s]
4948it [00:04, 1117.94it/s]
0it [00:00, ?it/s]
  0%|          | 0/12759 [00:00<?, ?B/s]
1237it [00:01, 1014.67it/s]
100%|██████████| 6185/6185 [00:00<00:00, 62228.09it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 1.3312 ||: 100%|██████████| 155/155 [00:05<00:00, 27.16it/s]
loss: 0.7402 ||: 100%|██████████| 39/39 [00:01<00:00, 29.73it/s]
loss: 0.6963 ||: 100%|██████████| 155/155 [00:05<00:00, 25.64it/s]
loss: 0.6503 ||: 100%|██████████| 39/39 [00:01<00:00, 32.29it/s]
loss: 0.6335 ||: 100%|██████████| 155/155 [00:05<00:00, 29.67it/s]
loss: 0.6063 ||: 100%|██████████| 39/39 [00:01<00:00, 32.95it/s]
loss: 0.5533 ||: 100%|██████████| 155/155 [00:05<00:00, 27.92it/s]
loss: 0.4903 ||: 100%|██████████| 39/39 [00:01<00:00, 31.49it/s]
loss: 0.4148 ||: 100%|██████████| 155/155 [00:05<00:00, 27.03it/s]
loss: 0.3565 ||: 100%|██████████| 39/39 [00:01<00:00, 29.19it/s]
loss:

{'best_epoch': 39, 'peak_cpu_memory_MB': 2758.048, 'peak_gpu_0_memory_MB': 853, 'training_duration': '0:12:57.897538', 'training_start_epoch': 0, 'training_epochs': 99, 'epoch': 99, 'training_loss': 0.013265338929276397, 'training_cpu_memory_MB': 2758.048, 'training_gpu_0_memory_MB': 853, 'validation_loss': 0.02274573089753409, 'best_validation_loss': 0.016938261175640106}


In [0]:
train_reader_4 = LTLSeq2SeqReader("https://raw.githubusercontent.com/CobiELF/ltl-amdp/master/mod/TRAIN_4.tsv")
test_reader_4 = LTLSeq2SeqReader("https://raw.githubusercontent.com/CobiELF/ltl-amdp/master/mod/TEST_4.tsv")
model_4 = LTLSeq2SeqModel(train_reader_4, test_reader_4)

print(str(model_4.metrics))

0it [00:00, ?it/s]
  0%|          | 0/48490 [00:00<?, ?B/s]
4948it [00:04, 1147.13it/s]
0it [00:00, ?it/s]
  0%|          | 0/12889 [00:00<?, ?B/s]
1237it [00:01, 991.74it/s]
100%|██████████| 6185/6185 [00:00<00:00, 52051.46it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 1.3250 ||: 100%|██████████| 155/155 [00:06<00:00, 25.52it/s]
loss: 0.7554 ||: 100%|██████████| 39/39 [00:01<00:00, 22.29it/s]
loss: 0.6908 ||: 100%|██████████| 155/155 [00:05<00:00, 26.00it/s]
loss: 0.6469 ||: 100%|██████████| 39/39 [00:01<00:00, 28.01it/s]
loss: 0.6108 ||: 100%|██████████| 155/155 [00:05<00:00, 26.42it/s]
loss: 0.5651 ||: 100%|██████████| 39/39 [00:01<00:00, 30.72it/s]
loss: 0.5180 ||: 100%|██████████| 155/155 [00:05<00:00, 26.41it/s]
loss: 0.4463 ||: 100%|██████████| 39/39 [00:01<00:00, 30.71it/s]
loss: 0.3955 ||: 100%|██████████| 155/155 [00:06<00:00, 25.66it/s]
loss: 0.3529 ||: 100%|██████████| 39/39 [00:01<00:00, 31.91it/s]
loss: 

{'best_epoch': 33, 'peak_cpu_memory_MB': 2804.844, 'peak_gpu_0_memory_MB': 893, 'training_duration': '0:12:19.943281', 'training_start_epoch': 0, 'training_epochs': 99, 'epoch': 99, 'training_loss': 0.014274640604825466, 'training_cpu_memory_MB': 2804.844, 'training_gpu_0_memory_MB': 893, 'validation_loss': 0.021368834920558116, 'best_validation_loss': 0.01602471635142604}


In [0]:
train_reader_5 = LTLSeq2SeqReader("https://raw.githubusercontent.com/CobiELF/ltl-amdp/master/mod/TRAIN_5.tsv")
test_reader_5 = LTLSeq2SeqReader("https://raw.githubusercontent.com/CobiELF/ltl-amdp/master/mod/TEST_5.tsv")
model_5 = LTLSeq2SeqModel(train_reader_5, test_reader_5)

print(str(model_5.metrics))

0it [00:00, ?it/s]
  0%|          | 0/48452 [00:00<?, ?B/s]
4948it [00:04, 1085.96it/s]
0it [00:00, ?it/s]
  0%|          | 0/12938 [00:00<?, ?B/s]
1237it [00:01, 1006.65it/s]
100%|██████████| 6185/6185 [00:00<00:00, 55888.51it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
loss: 1.3067 ||: 100%|██████████| 155/155 [00:05<00:00, 26.50it/s]
loss: 0.7422 ||: 100%|██████████| 39/39 [00:01<00:00, 32.16it/s]
loss: 0.7192 ||: 100%|██████████| 155/155 [00:05<00:00, 27.74it/s]
loss: 0.6461 ||: 100%|██████████| 39/39 [00:01<00:00, 32.60it/s]
loss: 0.6176 ||: 100%|██████████| 155/155 [00:05<00:00, 28.57it/s]
loss: 0.5740 ||: 100%|██████████| 39/39 [00:01<00:00, 31.57it/s]
loss: 0.5302 ||: 100%|██████████| 155/155 [00:05<00:00, 26.38it/s]
loss: 0.4739 ||: 100%|██████████| 39/39 [00:01<00:00, 31.87it/s]
loss: 0.4069 ||: 100%|██████████| 155/155 [00:05<00:00, 29.07it/s]
loss: 0.3677 ||: 100%|██████████| 39/39 [00:01<00:00, 33.63it/s]
loss:

{'best_epoch': 25, 'peak_cpu_memory_MB': 2851.652, 'peak_gpu_0_memory_MB': 893, 'training_duration': '0:13:34.814923', 'training_start_epoch': 0, 'training_epochs': 99, 'epoch': 99, 'training_loss': 0.013852697788400203, 'training_cpu_memory_MB': 2851.652, 'training_gpu_0_memory_MB': 893, 'validation_loss': 0.025540270237448006, 'best_validation_loss': 0.021562413137871772}


In [0]:
_, _ = evaluate(model_1, test_reader_1, "results_1.tsv")
_, _ = evaluate(model_2, test_reader_2, "results_2.tsv")
_, _ = evaluate(model_3, test_reader_3, "results_3.tsv")
_, _ = evaluate(model_4, test_reader_4, "results_4.tsv")
_, _ = evaluate(model_5, test_reader_5, "results_5.tsv")

ACC: %89.08649959579628
TOPK: %100.0
ACC: %88.03556992724333
TOPK: %100.0
ACC: %89.97574777687954
TOPK: %99.91915925626516
ACC: %91.18835893290218
TOPK: %99.91915925626516
ACC: %92.32012934518997
TOPK: %100.0


In [0]:
# class ConstrainedLTLDecoder(nn.Module): # most code borrowed from 585
#   def __init__(self, d_word, d_hid, len_voc):

#     super(ConstrainedLTLDecoder, self).__init__()
#     self.d_word = d_word
#     self.d_hid = d_hid
#     self.len_voc = len_voc

#     # embeddings
#     self.embs = nn.Embedding(len_voc, d_word)

#     # RNN-RNN encoder decoder
#     self.encoder = nn.RNN(d_word, d_hid, batch_first=True)
#     self.decoder = nn.RNN(d_word, d_hid, batch_first=True)
    
#     # output layer (softmax will be applied after this)
#     self.out = nn.Linear(d_hid, len_voc)

#     # perform forward propagation of S2S model
#     def forward(self, inputs, outputs):
        
#       bsz, max_len = inputs.size()
#       e_embs = self.embs(inputs)
#       _, e_hid = self.encoder(e_embs)
      
#       d_embs = self.embs(outputs)
#       d_hids, _ = self.decoder(d_embs, e_hid)
#       d_hids = d_hids.squeeze(0)

#       decoder_preds = self.out(d_hids)
#       decoder_preds = torch.nn.functional.log_softmax(decoder_preds, dim=1)

#       return decoder_preds

#     # given a previous word and a previous hidden state
#     # produce a probability distribution over the entire vocabulary
#     def single_decoder_step(self, prev_word, prev_hid):
        
#       if prev_word is not None:
#           decoder_input = self.embs(prev_word).view(1, 1, self.d_word)
#       else:
#           decoder_input = torch.zeros(1, 1, self.d_word)

#       # feed prev hidden state and prev char to decoder to get current hidden state
#       _, hn = self.decoder(decoder_input, prev_hid)

#       # feed into output layer and apply softmax to get probability distribution over vocab
#       pred_dist = self.out(hn.view(-1, self.d_hid))
#       pred_dist = torch.nn.functional.log_softmax(pred_dist, dim=1)
#       return pred_dist.view(-1), hn

#     # greedy search for one input sequence (bsz = 1)
#     def greedy_search(self, seq):

#       #TODO do this with beam search

#       # grammar goes here

#       bsz, max_len = seq.size()

#       output_seq = [] # this will contain our output sequence
#       output_prob = 0. # and this will be the probability of that sequence

#       # get embeddings of inputs
#       embs = self.embs(seq)

#       # encode input sentence and extract final hidden state of encoder RNN
#       _, hn = self.encoder(embs)
#       prev_word = None
      
#       # now decode one character at a time
#       for idx in range(15):

#           pred_dist, hn = self.single_decoder_step(prev_word, hn)

#           # in greedy search, we will just use the argmax prediction at each time step
#           # argmax_pred = torch.argmax(pred_dist)
#           # argmax_prob = pred_dist[argmax_pred]
#           output_seq.append(find_argmax_in_subset(pred_dist, trans[prev_word]))
#           # now use the argmax prediction as the input to the next time step
#           prev_word = argmax_pred
          
#           # stop if EOS
#           if idx_to_w[int(argmax_pred)] == '<EOS>':
#               break

#       return output_seq